In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import tensorflow.keras
import os
checkpoint_path = "training_ResNet_52/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
from keras.callbacks import History

In [3]:
d_rate=0.1 #The rate of droupout for the layers in the ResNet
act='gelu'#The activation function use ('relu','swish','elu','gelu','selu')
lr=0.01 #The default learning rate Default 0.01
mom=0.9 #The momentum parameter Default 0.9
test_split=1 #Default 1 and range[1,0)
#Reducing test_split will make the model faster but will also reduce the accuracy of the Validation metric
#On changing test_split run the lower initialisation cell, marked below**

In [4]:
df_data=pd.read_csv("DataSetNormalised.csv")

df_data

In [5]:
df_label=pd.read_csv("LabelSet.csv")

In [6]:
x=np.array(df_data.iloc[:,1:])

In [7]:
y=np.array(df_label.iloc[:,1])

In [8]:
x=x.astype(np.float32)

In [9]:
#On changing test_split run this--------**
size=10000000 + test_split*1000000
x_train=x[:][:10000000]
y_train=y[:][:10000000]
x_test=x[:][10000000:size]
y_test=y[:][10000000:size]

y_train.shape

In [10]:
def identity_block(X,no_nodes,Activation):#A custom Residual identity block with a skip connection
    X_shortcut=X
    #There will be 3 layers in total for the sequential connection
    #Activation will be specified by the user as per the Keras Documentation
    #no_nodes denote the size of the model which will be built
    #Applying Batch Normalisation to size the inputs
    #Layer 1 -----------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 2-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 3-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 4-------------------------------------------------
    X= tf.keras.layers.Add()([X_shortcut, X])
    X=tf.keras.layers.Activation('relu')(X)
    #--------------------------------------------------------
    X_shortcut2=X
     #Layer 5 -----------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 6-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 7-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    X= tf.keras.layers.Add()([X_shortcut, X,X_shortcut2])
    #Layer 8-------------------------------------------------
    X=tf.keras.layers.Activation('relu')(X)
    return X
    

In [11]:
inputs=tf.keras.layers.Input(shape=(28,))                    #Keras input layer
X=tf.keras.layers.Dense(300,activation=act)(inputs)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)        
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(X)     #keras output layer
model=tf.keras.Model(inputs,outputs)                         #Model declaration

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 300)          8700        ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 300)          90300       ['dense[0][0]']                  
                                                                                                  
 batch_normalization (BatchNorm  (None, 300)         1200        ['dense_1[0][0]']                
 alization)                                                                                   

In [13]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.01,
        beta_1=mom,
        beta_2=0.999),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC']
    )

In [14]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)



Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x00000173BEA9ACD0> and <keras.layers.core.dropout.Dropout object at 0x00000173BEB6DBE0>).

Two checkpoint references resolved to different objects (<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000173BF0B6A90> and <keras.layers.core.dense.Dense object at 0x00000173BEA9ACD0>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x00000173BF0C2E50> and <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000173BF0B6A90>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x00000173BF0DA790> and <keras.layers.core.dropout.Dropout object at 0x00000173BF0CB790>).

Two checkpoint references resolved to different objects (<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000173BF0DF6D0> and <keras.layers.mer


Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x00000173BF13FCD0> and <keras.layers.merge.Add object at 0x00000173BF104820>).

Two checkpoint references resolved to different objects (<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000173BF12E670> and <keras.layers.core.activation.Activation object at 0x00000173BF12E700>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x00000173BF153730> and <keras.layers.core.dense.Dense object at 0x00000173BF132280>).

Two checkpoint references resolved to different objects (<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000173BF0EBC40> and <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000173BF1371C0>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x00000173BF13F220> and <keras.layers.core.dro


Two checkpoint references resolved to different objects (<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000173BF2EEB80> and <keras.layers.core.dropout.Dropout object at 0x00000173BF109FD0>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x00000173BF2DB640> and <keras.layers.core.dense.Dense object at 0x00000173BF0DFBB0>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x00000173BF2E6E20> and <keras.layers.core.dense.Dense object at 0x00000173BF1046D0>).

Two checkpoint references resolved to different objects (<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000173BF2BAE80> and <keras.layers.core.dropout.Dropout object at 0x00000173BF16A310>).

Two checkpoint references resolved to different objects (<keras.layers.core.dense.Dense object at 0x00000173BF2F85B0> and <keras.layers.merge.Add object at 0x00000173BF16A8B0>

In [15]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=True,
                                                 save_best_only=True)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.5,
                                                 patience=5,
                                                 min_lr=10**-6,
                                                 cooldown=5)

history = History()

model.fit(
    x_train,
    y_train,
    epochs=300,
    batch_size=10240,
    validation_data=(x_test,y_test),
    callbacks=[cp_callback, reduce_lr, history]
)

Epoch 1/300


InternalError: 2 root error(s) found.
  (0) INTERNAL:  Attempting to perform BLAS operation using StreamExecutor without BLAS support
	 [[node model/dense/MatMul
 (defined at C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\layers\core\dense.py:199)
]]
	 [[assert_greater_equal/Assert/AssertGuard/pivot_f/_3/_93]]
  (1) INTERNAL:  Attempting to perform BLAS operation using StreamExecutor without BLAS support
	 [[node model/dense/MatMul
 (defined at C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\layers\core\dense.py:199)
]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_14362]

Errors may have originated from an input operation.
Input Source operations connected to node model/dense/MatMul:
In[0] IteratorGetNext (defined at C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\engine\training.py:866)	
In[1] model/dense/MatMul/ReadVariableOp:

Operation defined at: (most recent call last)
>>>   File "C:\Users\ML_RTC\anaconda3\lib\runpy.py", line 194, in _r

In [ ]:
model.save('ResNet_52')

with open("sunnyMamoni.txt", "w") as f:
    f.write(str(history.history))